In [1]:
%%capture

!pip install ipython-autotime

%load_ext autotime

time: 1.93 ms (started: 2022-04-23 20:47:39 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Package Installations

In [2]:
%%capture
!pip install kaggle

time: 2.53 s (started: 2022-04-23 20:47:39 +00:00)


In [3]:
%%capture
!pip install transformers

time: 7.66 s (started: 2022-04-23 20:47:42 +00:00)


In [4]:
%%capture
!pip install -U torchtext==0.8.0

time: 4.64 s (started: 2022-04-23 20:47:49 +00:00)


In [5]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import random
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm.notebook import tqdm
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import transformers
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

time: 10.5 s (started: 2022-04-23 20:47:54 +00:00)


In [6]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

time: 1.45 s (started: 2022-04-23 20:48:05 +00:00)


In [7]:
seed_val = 500
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

time: 4.61 ms (started: 2022-04-23 20:48:06 +00:00)


Kaggle Dataset

In [8]:
# Create the kaggle directory and read the uploaded kaggle.json file
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

time: 249 ms (started: 2022-04-23 20:48:06 +00:00)


In [9]:
!chmod 600 /root/.kaggle/kaggle.json

time: 124 ms (started: 2022-04-23 20:48:06 +00:00)


In [10]:
# Download dataset
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

 81% 33.0M/41.0M [00:01<00:00, 30.7MB/s]
100% 41.0M/41.0M [00:01<00:00, 32.0MB/s]
time: 3.13 s (started: 2022-04-23 20:48:06 +00:00)


In [11]:
# Unzip folder in Colab content folder
!unzip /content/fake-and-real-news-dataset.zip

Archive:  /content/fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                
time: 1.14 s (started: 2022-04-23 20:48:10 +00:00)


In [12]:
df1 = pd.read_csv("Fake.csv")
df2 = pd.read_csv("True.csv")
df1["y"] = ["False"]*len(df1)
df2["y"] = ["True"]*len(df2)

time: 1.73 s (started: 2022-04-23 20:48:11 +00:00)


In [13]:
frames = [df1, df2]
corpus = pd.concat(frames, ignore_index=True)

time: 11.1 ms (started: 2022-04-23 20:48:12 +00:00)


In [14]:
corpus["x"] = corpus["title"]

time: 6.59 ms (started: 2022-04-23 20:48:13 +00:00)


In [15]:
y_encoder = LabelEncoder()
corpus['y'] = y_encoder.fit_transform(corpus['y'])

time: 27.9 ms (started: 2022-04-23 20:48:13 +00:00)


In [16]:
corpus = corpus[["x", "y"]]

time: 22 ms (started: 2022-04-23 20:48:13 +00:00)


In [17]:
corpus.drop_duplicates(inplace=True)
corpus

,x,y
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0
...,...,...
44892,North Korea shipments to Syria chemical arms a...,1
44894,LexisNexis withdrew two products from Chinese ...,1
44895,Minsk cultural hub becomes haven from authorities,1
44896,Vatican upbeat on possibility of Pope Francis ...,1


time: 62.8 ms (started: 2022-04-23 20:48:13 +00:00)


**Reddit Dataset**

In [18]:
reddit_df = pd.read_csv('/content/news_posts.csv')
reddit_df

,Unnamed: 0,Title,Post Text,ID,Score,Total Comments,Post URL
0,0,People Are Accusing Robinhood Of Stealing From...,NaN,l7afyx,181017,4408,https://www.buzzfeednews.com/article/clarissaj...
1,1,US Military Could Lose Space Force Trademark t...,NaN,gyzw2p,129257,2844,https://www.cbr.com/us-military-lose-space-for...
2,2,White House threatens to fire anyone who tries...,NaN,jrskag,126468,4141,https://americanindependent.com/white-house-th...
3,3,Meta's threat to close down Facebook and Insta...,NaN,so0ree,126258,3786,https://www.cityam.com/metas-threat-to-close-d...
4,4,Don't eat or inject yourself with disinfectant...,NaN,g6zci5,125437,7024,https://www.cnn.com/world/live-news/coronaviru...
...,...,...,...,...,...,...,...
4898,987,Colorado pizza delivery driver saves mans life...,NaN,3b37gf,24221,898,http://www.postindependent.com/news/16943384-1...
4899,988,"Utah Woman Donates Over 1,200 Handmade Toys to...",NaN,90pbng,24216,888,https://www.insideedition.com/utah-woman-donat...
4900,989,Year of the Tiger marks increase in tiger popu...,NaN,sinyqg,24197,205,https://democratic-europe.eu/2022/02/01/%ef%bf...
4901,990,Florida cops deliver dresses made by a 99-year...,NaN,8q8njr,24164,183,http://www.miamiherald.com/news/state/florida/...


time: 51.3 ms (started: 2022-04-23 20:48:13 +00:00)


In [19]:
reddit_df['x'] = reddit_df['Title']

time: 1.72 ms (started: 2022-04-23 20:48:13 +00:00)


In [20]:
reddit_df['y'] = ["True"]*4903
reddit_df['y'] = y_encoder.transform(reddit_df['y'])

time: 5.31 ms (started: 2022-04-23 20:48:13 +00:00)


In [21]:
reddit_df = reddit_df[["x", "y"]]
reddit_df.drop_duplicates(inplace=True)
reddit_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,People Are Accusing Robinhood Of Stealing From...,1
1,US Military Could Lose Space Force Trademark t...,1
2,White House threatens to fire anyone who tries...,1
3,Meta's threat to close down Facebook and Insta...,1
4,Don't eat or inject yourself with disinfectant...,1
...,...,...
4898,Colorado pizza delivery driver saves mans life...,1
4899,"Utah Woman Donates Over 1,200 Handmade Toys to...",1
4900,Year of the Tiger marks increase in tiger popu...,1
4901,Florida cops deliver dresses made by a 99-year...,1


time: 36.7 ms (started: 2022-04-23 20:48:13 +00:00)


**Twitter Dataset**

In [22]:
twitter_real_df = pd.read_csv('/content/real_news_tweets.csv')
twitter_real_df

,Unnamed: 0,Datetime,Tweet Id,Text,Username
0,0,2022-04-19 08:35:25+00:00,1516334641610989571,Half-a-million Ukrainians have fled to the Lvi...,AP
1,1,2022-04-19 07:42:46+00:00,1516321393281015809,Afghan police say explosions targeting educati...,AP
2,2,2022-04-19 07:13:55+00:00,1516314133666349058,Russian forces have attacked along a broad fro...,AP
3,3,2022-04-19 06:40:38+00:00,1516305757674819584,There's stark public divide in Iran over Russi...,AP
4,4,2022-04-19 06:26:26+00:00,1516302180772364288,There's stark public divide in Iran over Russi...,AP
...,...,...,...,...,...
4995,4995,2022-02-24 00:13:22+00:00,1496639353820430337,The soldiers on the front line of the Ukraine-...,BBCWorld
4996,4996,2022-02-23 23:45:08+00:00,1496632246459158531,What's Putin's next move?,BBCWorld
4997,4997,2022-02-23 23:29:06+00:00,1496628212880875520,"Virgin Hyperloop, a futuristic train service e...",BBCWorld
4998,4998,2022-02-23 23:14:06+00:00,1496624437243879429,Metaverse app allows kids into virtual strip c...,BBCWorld


time: 72.6 ms (started: 2022-04-23 20:48:13 +00:00)


In [23]:
twitter_fake_df = pd.read_csv('/content/fake_news_tweets.csv')
twitter_fake_df

,Unnamed: 0,Datetime,Tweet Id,Text,Username
0,0,2022-04-19 08:30:02+00:00,1516333287744102405,Grandpa Surprisingly Willing To Talk About Man...,TheOnion
1,1,2022-04-19 06:30:01+00:00,1516303084628099076,Nation Attempts To Fall Asleep By Doing Little...,TheOnion
2,2,2022-04-19 05:00:01+00:00,1516280436334473221,"When it comes to pip blips, Megan Fox takes th...",TheOnion
3,3,2022-04-19 04:30:01+00:00,1516272886314672130,Longtime Sleepytime Tea Addict Has To Use 6 Ba...,TheOnion
4,4,2022-04-19 03:15:00+00:00,1516254005869096968,The Onion: Now on another part of your phone. ...,TheOnion
...,...,...,...,...,...
4995,4995,2021-10-02 14:00:00+00:00,1444301140703780868,Boss Warns All New Employees He Will Fire Them...,TheBabylonBee
4996,4996,2021-10-02 13:00:02+00:00,1444286051204403205,10 Tips For Making Sure Your Kids Are Terrifie...,TheBabylonBee
4997,4997,2021-10-02 12:21:32+00:00,1444276362865479696,Biden Says We Can Afford $3.5T Bill Because Ch...,TheBabylonBee
4998,4998,2021-10-02 02:00:00+00:00,1444119947274334211,Fly That Landed On Mike Pence During Debate Re...,TheBabylonBee


time: 52.4 ms (started: 2022-04-23 20:48:13 +00:00)


In [24]:
twitter_real_df['x'] = twitter_real_df['Text']
twitter_fake_df['x'] = twitter_fake_df['Text']

time: 2.71 ms (started: 2022-04-23 20:48:13 +00:00)


In [25]:
twitter_real_df['y'] = ['True'] * 5000
twitter_real_df['y'] = y_encoder.transform(twitter_real_df['y'])

time: 5.9 ms (started: 2022-04-23 20:48:13 +00:00)


In [26]:
twitter_fake_df['y'] = ['False'] * 5000
twitter_fake_df['y'] = y_encoder.transform(twitter_fake_df['y'])

time: 6.4 ms (started: 2022-04-23 20:48:13 +00:00)


In [27]:
twitter_real_df = twitter_real_df[["x", "y"]]
twitter_real_df.drop_duplicates(inplace=True)
twitter_real_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,Half-a-million Ukrainians have fled to the Lvi...,1
1,Afghan police say explosions targeting educati...,1
2,Russian forces have attacked along a broad fro...,1
3,There's stark public divide in Iran over Russi...,1
4,There's stark public divide in Iran over Russi...,1
...,...,...
4995,The soldiers on the front line of the Ukraine-...,1
4996,What's Putin's next move?,1
4997,"Virgin Hyperloop, a futuristic train service e...",1
4998,Metaverse app allows kids into virtual strip c...,1


time: 30.4 ms (started: 2022-04-23 20:48:13 +00:00)


In [28]:
twitter_fake_df = twitter_fake_df[["x", "y"]]
twitter_fake_df.drop_duplicates(inplace=True)
twitter_fake_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,Grandpa Surprisingly Willing To Talk About Man...,0
1,Nation Attempts To Fall Asleep By Doing Little...,0
2,"When it comes to pip blips, Megan Fox takes th...",0
3,Longtime Sleepytime Tea Addict Has To Use 6 Ba...,0
4,The Onion: Now on another part of your phone. ...,0
...,...,...
4995,Boss Warns All New Employees He Will Fire Them...,0
4996,10 Tips For Making Sure Your Kids Are Terrifie...,0
4997,Biden Says We Can Afford $3.5T Bill Because Ch...,0
4998,Fly That Landed On Mike Pence During Debate Re...,0


time: 23.9 ms (started: 2022-04-23 20:48:13 +00:00)


In [29]:
from sklearn import model_selection
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

time: 4.54 ms (started: 2022-04-23 20:48:13 +00:00)


In [30]:
frame1 = [twitter_real_df, twitter_fake_df]
corpus_tweet = pd.concat(frame1)
corpus_tweet

,x,y
0,Half-a-million Ukrainians have fled to the Lvi...,1
1,Afghan police say explosions targeting educati...,1
2,Russian forces have attacked along a broad fro...,1
3,There's stark public divide in Iran over Russi...,1
4,There's stark public divide in Iran over Russi...,1
...,...,...
4995,Boss Warns All New Employees He Will Fire Them...,0
4996,10 Tips For Making Sure Your Kids Are Terrifie...,0
4997,Biden Says We Can Afford $3.5T Bill Because Ch...,0
4998,Fly That Landed On Mike Pence During Debate Re...,0


time: 28.9 ms (started: 2022-04-23 20:48:13 +00:00)


**XLNet**

In [31]:
%%capture
pip install simpletransformers

time: 33.5 s (started: 2022-04-23 20:48:13 +00:00)


In [32]:
from simpletransformers.classification import ClassificationModel


# Create a ClassificationModel
model =ClassificationModel('xlnet', 'xlnet-base-cased', num_labels=2)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

time: 35.6 s (started: 2022-04-23 20:48:47 +00:00)


In [33]:
# Create a TransformerModel with modified attributes
from simpletransformers.model import TransformerModel
model = TransformerModel('xlnet', 'xlnet-base-cased', num_labels=2, args={'learning_rate':1e-5, 'num_train_epochs': 2, 'reprocess_input_data': True, 'overwrite_output_dir': True})

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

time: 9.25 s (started: 2022-04-23 20:49:22 +00:00)


In [34]:
train = corpus
frame2 = [reddit_df, corpus_tweet]
test = pd.concat(frame2)
test

,x,y
0,People Are Accusing Robinhood Of Stealing From...,1
1,US Military Could Lose Space Force Trademark t...,1
2,White House threatens to fire anyone who tries...,1
3,Meta's threat to close down Facebook and Insta...,1
4,Don't eat or inject yourself with disinfectant...,1
...,...,...
4995,Boss Warns All New Employees He Will Fire Them...,0
4996,10 Tips For Making Sure Your Kids Are Terrifie...,0
4997,Biden Says We Can Afford $3.5T Bill Because Ch...,0
4998,Fly That Landed On Mike Pence During Debate Re...,0


time: 20.5 ms (started: 2022-04-23 20:49:32 +00:00)


In [35]:
model.train_model(train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/38729 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/4842 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 2:   0%|          | 0/4842 [00:00<?, ?it/s]

(9684, 0.01726895547058404)

time: 23min 25s (started: 2022-04-23 20:49:32 +00:00)


In [36]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/13820 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1728 [00:00<?, ?it/s]

time: 1min 12s (started: 2022-04-23 21:12:58 +00:00)


In [38]:
from pprint import pprint
pprint(result)

{'acc': 0.882561505065123,
 'auprc': 0.973227716777118,
 'auroc': 0.9387485770748142,
 'eval_loss': 1.1056322135997039,
 'f1': 0.8825615050651229,
 'fn': 1197,
 'fp': 426,
 'mcc': 0.7406056269120102,
 'tn': 3727,
 'tp': 8470}
time: 1.57 ms (started: 2022-04-23 21:14:10 +00:00)


In [ ]:
save_model_dir='xlnet_kaggle_2_epochs'

In [ ]:
src_dir = 'outputs'

In [ ]:
!mkdir {save_model_dir}

In [ ]:
!cp {src_dir}/*.* {save_model_dir}

In [ ]:
!rm -r {src_dir}

In [ ]:
zip_filename = save_model_dir + '.zip'

In [ ]:
! zip -r {zip_filename} {save_model_dir}

In [ ]:
!cp {zip_filename} '/content/gdrive/MyDrive/'